## Library importing

In [1]:
%run libs.py
%run helper_funcs.py
%run callback.py

## Data preparation

In [2]:
cc = 'ADA'
df = read_data(cc)
df.head(2)

,unix,date,symbol,open,high,low,close,Volume USD,Volume ADA
0,1652400000000,2022-05-13 00:00:00,ADA/USD,0.525175,0.547533,0.520565,0.547533,4842.449232,8844.128714
1,1652396400000,2022-05-12 23:00:00,ADA/USD,0.513225,0.530541,0.513225,0.525018,7690.348439,14647.783600


In [3]:
df = reformat_data(df, cc)

NaNs in our data:

unix          0
date          0
symbol        0
open          0
high          0
low           0
close         0
Volume USD    0
Volume ADA    0
dtype: int64

The reformatted data:
                         Open      High       Low     Close        Volume
date                                                                     
2022-05-13 00:00:00  0.525175  0.547533  0.520565  0.547533   8844.128714
2022-05-12 23:00:00  0.513225  0.530541  0.513225  0.525018  14647.783600


In [4]:
print(((df == 0).sum(axis=1) > 0).sum() / len(df))

df_2 = df.copy().loc[(df == 0).sum(axis=1) == 0, :]
print(df_2.head(2))
print(len(df_2), len(df))

0.07745951242003343
                         Open      High       Low     Close        Volume
date                                                                     
2022-05-13 00:00:00  0.525175  0.547533  0.520565  0.547533   8844.128714
2022-05-12 23:00:00  0.513225  0.530541  0.513225  0.525018  14647.783600
32014 34702


In [5]:
train_df, test_df = prepare_traintest_data(df_2, 3/4)

Length of training data: 24010

Length of test data: 8004


## Trading by random guesses

In [6]:
window_size = 24
env = gym.make('stocks-v0', df=test_df, frame_bound=(24, len(test_df)), window_size=window_size)
env.signal_features[: 2, :]

array([[0.05933, 0.     ],
       [0.0601 , 0.00077]])

In [7]:
env.action_space

Discrete(2)

In [17]:
state = env.reset()
while True: 
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    if done: 
        print("info", info)
        break

info {'total_reward': -0.05702000000000004, 'total_profit': 3.291893018509061e-14, 'position': 1}


## Hyperparameters to test

In [4]:
split_ratio = 3 / 4
log_dir = 'gym-anytrading/'

combs = [[[64], [64]],
         [[128], [128]],
         [64, [64], [64]]]
timestep_vals = [1000, 10000, 100000, 1000000]
policies = ['A2C', 'PPO']
ccs = ['ADA', 'BTC', 'ETH']
window_sizes = [1, 3, 6, 12, 24]

hps = [combs, timestep_vals, policies, window_sizes]
hp_combs = list(itertools.product(*hps))

In [5]:
for cc in ccs:
    df = read_data(cc)
    df = reformat_data(df, cc)
    df_2 = df.copy().loc[(df == 0).sum(axis=1) == 0, :]
    
    train_df, test_df = prepare_traintest_data(df_2, split_ratio)
    
    for (comb, timesteps, policy, window_size) in hp_combs:
        print(comb, timesteps, policy, window_size)
        test_hp_set(cc, train_df, test_df, window_size, log_dir, comb, policy, 0, timesteps)

NaNs in our data:

unix          0
date          0
symbol        0
open          0
high          0
low           0
close         0
Volume USD    0
Volume BTC    0
dtype: int64

The reformatted data:
                         Open     High       Low     Close    Volume
date                                                                
2022-05-13 00:00:00  32231.07  32640.0  31963.53  32551.64  3.433873
2022-05-12 23:00:00  31936.20  32420.0  31936.20  32236.45  5.193502
Length of training data: 25998

Length of test data: 8666
[[64], [64]] 1000 A2C 1
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[64], [64]] 1000 A2C 3
performance: {'total_reward': -36.75999999999976, 'total_profit': 0.9038746066159514, 'position': 0}
[[64], [64]] 1000 A2C 6
performance: {'total_reward': -3063.5699999999997, 'total_profit': 1.2027976649091772, 'position': 1}
[[64], [64]] 1000 A2C 12
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[64], [64]] 1000 A2C 24


performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[128], [128]] 1000000 A2C 1
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[128], [128]] 1000000 A2C 3
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[128], [128]] 1000000 A2C 6
performance: {'total_reward': 0.0, 'total_profit': 1.2213625461926672, 'position': 1}
[[128], [128]] 1000000 A2C 12
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[128], [128]] 1000000 A2C 24
performance: {'total_reward': 0.0, 'total_profit': 1.2006681430033856, 'position': 1}
[[128], [128]] 1000000 PPO 1
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[[128], [128]] 1000000 PPO 3
performance: {'total_reward': 0.0, 'total_profit': 1.2127991489361702, 'position': 1}
[[128], [128]] 1000000 PPO 6
performance: {'total_reward': 0.0, 'total_profit': 1.2213625461926672, 'position': 1}
[[128], [128]] 1000000 PPO 12
performance: {'total_reward': 0.0, 't

performance: {'total_reward': 350.3300000000004, 'total_profit': 2.33803840076538e-06, 'position': 1}
[[64], [64]] 10000 PPO 1
performance: {'total_reward': 49.13000000000012, 'total_profit': 1.927959112381276e-06, 'position': 0}
[[64], [64]] 10000 PPO 3
performance: {'total_reward': -45.92999999999995, 'total_profit': 1.923348956048843, 'position': 1}
[[64], [64]] 10000 PPO 6
performance: {'total_reward': 272.42000000000024, 'total_profit': 0.061288828774050716, 'position': 1}
[[64], [64]] 10000 PPO 12
performance: {'total_reward': 470.9300000000004, 'total_profit': 4.355185978604652e-06, 'position': 1}
[[64], [64]] 10000 PPO 24
performance: {'total_reward': 479.49, 'total_profit': 2.3258700255211646e-06, 'position': 1}
[[64], [64]] 100000 A2C 1
performance: {'total_reward': 0.0, 'total_profit': 3.615498516768063, 'position': 1}
[[64], [64]] 100000 A2C 3
performance: {'total_reward': 0.0, 'total_profit': 3.6641350318471333, 'position': 1}
[[64], [64]] 100000 A2C 6
performance: {'total

performance: {'total_reward': 8.589999999999918, 'total_profit': 0.04420661406441909, 'position': 0}
[64, [64], [64]] 1000 PPO 12
performance: {'total_reward': 420.5799999999994, 'total_profit': 2.547620932284286e-09, 'position': 1}
[64, [64], [64]] 1000 PPO 24
performance: {'total_reward': 10.179999999999993, 'total_profit': 0.00013358826496413673, 'position': 0}
[64, [64], [64]] 10000 A2C 1
performance: {'total_reward': -74.87, 'total_profit': 2.7887827670359453, 'position': 1}
[64, [64], [64]] 10000 A2C 3
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[64, [64], [64]] 10000 A2C 6
performance: {'total_reward': 0.0, 'total_profit': 3.6335851440121276, 'position': 1}
[64, [64], [64]] 10000 A2C 12
performance: {'total_reward': 0.0, 'total_profit': 1.0, 'position': 0}
[64, [64], [64]] 10000 A2C 24
performance: {'total_reward': 104.71000000000002, 'total_profit': 2.7960566253576977e-08, 'position': 0}
[64, [64], [64]] 10000 PPO 1
performance: {'total_reward': 7.540